In [65]:
import numpy as np
import pandas as pd
import csv
import scipy.stats as st


In [66]:
import numpy as np

class Neural_Network():
    def __init__(self, data, output, hls=20) -> None:
        self.data = data
        self.y = output
        self.output_size = np.shape(output)[1]
        self.hiddenL_size = hls
        self.m = np.shape(data)[0]
        self.n = np.shape(data)[1]

        # Adding bias terms to weights
        self.W1 = np.random.uniform(-0.5, 0.5, (self.hiddenL_size, self.n))
        self.b1 = np.zeros((self.hiddenL_size, 1))  # Bias for the first layer
        self.W2 = np.random.uniform(-0.5, 0.5, (self.output_size, self.hiddenL_size))
        self.b2 = np.zeros((self.output_size, 1))  # Bias for the second layer

        self.regularization = 5
        self.loss = []

    def run(self, epochs):
        nr_correct = 0
        for i in range(epochs):
            print(f'epoch : {i}')
            for img, l in zip(self.data[:], self.y[:]):
                img.shape += (1,)
                l.shape += (1,)
                o = self.forward(img, l)

                error = (1 / len(o)) * np.sum((o - l) ** 2, axis=0)
                nr_correct += int(np.argmax(o) == np.argmax(l))

                self.backward(o, img, l)
            print(f"Acc: {round((nr_correct / self.data.shape[0]) * 100, 3)}%")
            print(" img shape ", np.shape(img))
            print(f" mean loss {np.mean(self.loss)}  error : {error}")
            self.loss = []
            nr_correct = 0

    def forward(self, img, y):
        self.z1 = np.dot(self.W1, img) + self.b1
        self.a1 = self.sigmoid(self.z1)
        self.z2 = np.dot(self.W2, self.a1) + self.b2
        self.a2 = self.sigmoid(self.z2)
        y_hat = self.a2

        return y_hat

    def backward(self, o, img, y):
        alpha = 0.01
        delta_o = (o - y) * self.sigmoidPrime(o)
        self.loss.append(np.mean(delta_o))

        delta_a1 = np.dot(self.W2.T, delta_o) * self.sigmoidPrime(self.a1)

        self.W2 += -alpha * (np.dot(delta_o, self.a1.T) + (self.regularization / self.m) * self.W2)
        self.b2 += -alpha * np.sum(delta_o, axis=1, keepdims=True)

        self.W1 += -alpha * (np.dot(delta_a1, img.T) + (self.regularization / self.m) * self.W1)
        self.b1 += -alpha * np.sum(delta_a1, axis=1, keepdims=True)

    def test(self, X):
        z1 = np.dot(X, self.W1) + self.b1
        a1 = self.sigmoid(z1)
        z2 = np.dot(a1, self.W2) + self.b2
        a2 = self.sigmoid(z2)
        y_hat = a2

        return y_hat

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def sigmoidPrime(self, a):
        return a * (1 - a)


In [67]:
from data import get_mnist
import matplotlib.pyplot as plt

images , labels = get_mnist()
split_per = int(len(images)*0.8)
train_images= images[:split_per]
train_labels= labels[:split_per]

test_images= images[split_per:]
test_labels= labels[split_per:]

hls =20
NN = Neural_Network(train_images,train_labels,hls)

NN.run(1)




48000
epoch : 0
Acc: 62.354%
 img shape  (784, 1)
 mean loss 0.0009422534996811346  error : [0.01752923]


In [68]:
while True:
            index = int(input("Enter a number (0 - 9999): "))
            img = test_images[index]
            plt.imshow(img.reshape(28, 28), cmap="Greys")

            img.shape += (1,)
            # Forward propagation input -> hidden
            z =  NN.W1 @ img.reshape(784, 1)
            a = 1 / (1 + np.exp(-z))
            # Forward propagation hidden -> output
            a2 = + NN.W2@ a
            o = 1 / (1 + np.exp(-a2))

            plt.title(f"Number guessed : {o.argmax()} ")
            plt.show()

ValueError: invalid literal for int() with base 10: ''

In [ ]:
import pickle
model={
    'nb_hiddenLayers':1,
    'hiddenLayerSize':hls,
    'weights1':NN.W1,
    'weights2':NN.W2,
}
arr = np.zeros((1000,2))
with open('model.pkl','wb') as f:
    pickle.dump(model, f)
    
with open('model.pkl','rb') as f:
   x = pickle.load(f)
   print(model)

{'nb_hiddenLayers': 1, 'hiddenLayerSize': 20, 'weights1': array([[ 0.08220378, -0.03870507,  0.33231006, ..., -0.05348161,
         0.07960643,  0.25302602],
       [ 0.02257691, -0.29334793,  0.40775677, ...,  0.4486583 ,
        -0.31742712,  0.47591137],
       [-0.02083948,  0.28078986, -0.36415015, ...,  0.1063555 ,
        -0.40213802,  0.35434814],
       ...,
       [-0.19843329,  0.09049158,  0.1424204 , ...,  0.42741497,
        -0.02309322, -0.25303581],
       [-0.20481182, -0.39348625, -0.47445171, ...,  0.019211  ,
         0.04180672,  0.41829698],
       [ 0.07130212,  0.29807033,  0.41087585, ..., -0.10638137,
        -0.40970121,  0.06618686]]), 'weights2': array([[-1.34574503,  0.85627803, -0.3151954 , -0.57014087, -2.88091703,
         0.58798439,  1.31273907, -1.80210596,  0.1335443 ,  0.23996243,
        -0.8541552 , -1.01065679, -1.16527485, -0.1711864 , -0.52841921,
        -0.84151654, -0.30851805,  0.70480397,  0.87284375,  0.53457118],
       [ 1.11509726, -0